In [3]:
import pandas as pd
from sklearn.utils import resample
import nlpaug.augmenter.word as naw
import os
import torch
from tqdm import tqdm
import random
import re  # <--- NEW: Imported regex module

# --- CONFIGURATION ---
INPUT_FILE = "robust_experience_training_data_unbalanced.csv"
OUTPUT_FILE = "robust_experience_training_data_bert_augmented.csv"
TARGET_PER_CLASS = 3333 

# --- "MAX STRENGTH" POISON SETTINGS ---
POISON_FT_INTERN_COUNT = 800       
POISON_FT_CONSULTANT_COUNT = 800   
POISON_FT_RESUME_FORMAT_COUNT = 500
POISON_FT_HARD_NEG_COUNT = 300     # <--- NEW: Inject "junior" tasks into Full-time

POISON_FL_INTERN_COUNT = 200       
BOOST_FL_SELF_EMPLOYED_COUNT = 500 

POISON_INT_SUMMER_COUNT = 500      
POISON_INT_HARD_NEG_COUNT = 300    # <--- NEW: Inject "high responsibility" into Internship

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on: {DEVICE}")

# --- NEW: TEXT CLEANING FUNCTION ---
def clean_text(text):
    """Removes dates, durations, and location headers to reduce bias."""
    text = str(text).lower()
    # Remove dates (e.g., Jan 2020, Sep 2015 - Aug 2019)
    text = re.sub(r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\.?\s?\d{2,4}', '', text)
    # Remove year ranges (e.g., 2015-2019)
    text = re.sub(r'\d{4}\s?-\s?\d{4}', '', text)
    # Remove "to present"
    text = re.sub(r'\d{4}\s?to\s?present', '', text)
    # Remove durations (e.g., "6 months")
    text = re.sub(r'\d+\s?(months|years)', '', text)
    # Remove city/state headers often found at start (e.g., "New York, NY |")
    text = re.sub(r'[a-z]+,\s?[a-z]{2}\s?\|', '', text) 
    return text.strip()

# --- LOAD BERT AUGMENTER ---
print("Loading BERT model for augmentation...")
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute", device=DEVICE, aug_p=0.3  
)

if not os.path.exists(INPUT_FILE):
    print(f"Error: {INPUT_FILE} not found.")
else:
    print("Loading data...")
    df = pd.read_csv(INPUT_FILE)
    
    # --- NEW: APPLY CLEANING BEFORE SPLITTING ---
    print("Applying Regex Cleaning (Removing dates/locations)...")
    df['text'] = df['text'].apply(clean_text)
    
    df_fulltime = df[df['label'] == 'Full-time']
    df_freelance = df[df['label'] == 'Freelance']
    df_intern = df[df['label'] == 'Internship']

    augmented_dfs = []

    # ==============================================================================
    # 2. FIX FULL-TIME (Expanded Prefixes & Suffixes)
    # ==============================================================================
    print(f"\n--- Processing Full-time ---")
    
    # A. "Manager & Coordinator" (800)
    print(f"   -> Injecting {POISON_FT_INTERN_COUNT} 'Manager' examples...")
    ft_intern_poison = df_fulltime.sample(n=POISON_FT_INTERN_COUNT, replace=True, random_state=42).copy()
    ft_intern_suffixes = [
        ". I also managed a team of three interns.",
        " and supervised summer interns on the project.",
        ". Responsibilities included reviewing intern code pull requests.",
        ". I mentored junior interns during their semester.",
        ". I started as an intern but was hired full-time after 3 months.",
        ". My role involved interviewing candidates for internship positions.",
        ". I oversaw the department's summer internship program.",
        ". Delegated low-priority tickets to the engineering interns.",
        ". Escalated complex issues that interns could not solve.",
        ". Provided technical leadership to junior staff and interns.",
        ". Promoted from intern to Associate, then to Senior Manager.",
        ". I organized the company-wide internship orientation.",
        " and served as the Program Coordinator for the intern class.",
        ". Managed the logistics for the summer internship program.",
        ". Designed and executed the internship training modules.",
        " acting as the main point of contact for the internship program.",
        ". Planned the curriculum for the incoming internship cohort.",
        " and facilitated the onboarding sessions for new interns.",
        ". I was the Internship Program Manager for the 2023 group.",
        ". Coordinated weekly workshops for the summer interns.",
        ". My role involved running the internship selection process.",
        ". Led the daily stand-up meetings for the intern team.",
        ". Corrected mistakes made by the previous intern.",
        ". Signed off on the final deliverables for the summer interns.",
        ". Conducted exit interviews for the departing interns.",
        ". Created the project roadmap for the engineering interns.",
        ". Acted as a buddy and mentor for the new intern hire.",
        ". Reviewed and approved timesheets for the internship staff.",
        ". Set performance goals for the seasonal interns.",
        ". Transitioned the intern's prototype into production code.",
        ". Answered technical questions from the developer interns.",
        ". Ensured the interns followed the company coding standards.",
        ". Assigned bugs and feature requests to the intern squad.",
        ". Provided feedback on the intern's design documents.",
        ". Managed the budget for the summer internship activities.",
        ". Organized social events for the full-time staff and interns.",
        ". Resolved conflicts between junior developers and interns.",
        ". Updated the documentation for the internship handbook.",
        ". Calibrated the performance ratings for the intern class.",
        ". Recommended the top performing intern for a full-time offer.",
        ". Scheduled training sessions for the marketing interns.",
        ". Oversaw the intern's research project from start to finish.",
        ". Checked the quality of data entered by the interns.",
        ". Explained the system architecture to the new interns.",
        ". Helped the intern troubleshoot their development environment.",
        ". Monitored the progress of the intern's capstone project.",
        ". Gave a presentation on career growth to the interns.",
        ". Facilitated code reviews for the software engineering interns.",
        ". Served as the primary supervisor for the design interns.",
        ". Managed the headcount allocation for the internship program."
    ]
    ft_intern_poison['text'] = ft_intern_poison['text'].apply(lambda x: str(x) + random.choice(ft_intern_suffixes))
    
    # B. "Full-time Consultant" (800)
    print(f"   -> Injecting {POISON_FT_CONSULTANT_COUNT} 'Full-time Consultant' examples...")
    ft_consultant_poison = df_fulltime.sample(n=POISON_FT_CONSULTANT_COUNT, replace=True, random_state=42).copy()
    ft_consultant_prefixes = [
        "Full-time Senior Consultant: ", "Permanent Strategy Consultant. ",
        "Internal Implementation Consultant. ", "Staff Consultant (Full-time). ",
        "Principal Consultant employed by the firm. ", "Lead Technical Consultant (Permanent Role). ",
        "Senior Solutions Consultant at Oracle. ", "Employed as a SAP Consultant. ",
        "Corporate Security Consultant (In-house). ", "Management Consultant (Full-time Staff). ",
        "Permanent Risk Consultant. ", "Full-time Recruitment Consultant. ",
        "Business Consultant (Internal Team). ", "Operations Consultant (Headquarters Staff). ",
        "Strategic Consultant reporting to the VP. ", "Technology Consultant (W2 Employee). ",
        "Financial Consultant - Permanent Position. ", "HR Consultant (Full-time). ",
        "Senior Cloud Consultant (Internal). ", "Junior Consultant (Salaried). ",
        "Associate Consultant (Full-time). ", "Managing Consultant (Permanent). ",
        "Cybersecurity Consultant (Staff Role). ", "Process Consultant (In-house). ",
        "Change Management Consultant (FTE). ", "ERP Consultant (Full-time). ",
        "Sales Consultant (Internal). ", "Marketing Consultant (Permanent Staff). ",
        "Legal Consultant (In-house Counsel). ", "Environmental Consultant (Full-time). ",
        "Software Consultant (Employee). ", "Data Consultant (Internal). ",
        "Network Consultant (Staff). ", "Infrastructure Consultant (Full-time). ",
        "Product Consultant (Permanent). ", "Design Consultant (In-house). ",
        "UX Consultant (Salaried Employee). ", "Project Consultant (Internal). ",
        "Quality Consultant (Full-time). ", "Compliance Consultant (Staff). ",
        "Retail Consultant (Permanent). ", "Energy Consultant (Full-time). ",
        "Logistics Consultant (Internal). ", "Supply Chain Consultant (FTE). ",
        "Education Consultant (Staff). ", "Training Consultant (Full-time). ",
        "Research Consultant (Internal). ", "Policy Consultant (Permanent). ",
        "Communications Consultant (In-house). ", "Media Consultant (Full-time). "
    ]
    ft_consultant_poison['text'] = ft_consultant_poison['text'].apply(lambda x: random.choice(ft_consultant_prefixes) + str(x))

    # C. "Resume Format" (500)
    print(f"   -> Injecting {POISON_FT_RESUME_FORMAT_COUNT} 'Resume Format' examples...")
    ft_resume_poison = df_fulltime.sample(n=POISON_FT_RESUME_FORMAT_COUNT, replace=True, random_state=42).copy()
    ft_resume_prefixes = [
        "New York, NY | Oct 2020 - Present | ", "San Francisco | Jan 2019 - Dec 2021 | ",
        "Remote | 2018 - Present | ", "London, UK • 5 Years Exp • ",
        "Full-time | Chicago, IL | ", "Austin, TX -- ", "Senior Role | 2015-2020 | ",
        "Seattle, WA | June 2016 - Aug 2019 | ", "Boston, MA - Full Time - ",
        "Los Angeles | 2021-Present | ", "Denver, CO • ", "Atlanta, GA | ",
        "Miami, FL -- ", "Toronto, ON | ", "Vancouver, BC • ",
        "Berlin, DE | ", "Paris, FR -- ", "Tokyo, JP | ",
        "Sydney, AU • ", "Singapore | ", "Dublin, IE -- ",
        "2015 - 2018 | ", "2019 - Present | ", "2020 - 2022 | ",
        "Jan 2017 - Feb 2020 | ", "March 2018 - April 2021 | ",
        "May 2019 - June 2022 | ", "July 2020 - Aug 2023 | ",
        "Sept 2021 - Oct 2024 | ", "Nov 2022 - Present | ",
        "Dec 2023 - Current | ", "Software Engineer | ",
        "Product Manager | ", "Data Scientist | ", "Designer | ",
        "Analyst | ", "Consultant | ", "Manager | ",
        "Director | ", "VP | ", "Lead | ", "Senior | ",
        "Junior | ", "Associate | ", "Staff | ", "Principal | ",
        "Head of | ", "Chief | ", "Founder | ", "Co-Founder | "
    ]
    ft_resume_poison['text'] = ft_resume_poison['text'].apply(lambda x: random.choice(ft_resume_prefixes) + str(x))

    # D. NEW: Hard Negatives (Full-time that sounds like Intern)
    print(f"   -> Injecting {POISON_FT_HARD_NEG_COUNT} 'Hard Negative' examples (Full-time Junior)...")
    ft_hard_neg_poison = df_fulltime.sample(n=POISON_FT_HARD_NEG_COUNT, replace=True, random_state=42).copy()
    ft_hard_neg_prefixes = [
        "Junior Associate learning the codebase. ",
        "Assisted the Senior Engineer with documentation. ",
        "Temporary 12-month contract converted to permanent. ",
        "Entry-level role involving basic data entry and support. ",
        "Shadowed the team lead to learn agile processes. ",
        "Junior Support Staff. ",
        "New Grad rotation program (Full-time). "
    ]
    ft_hard_neg_poison['text'] = ft_hard_neg_poison['text'].apply(lambda x: random.choice(ft_hard_neg_prefixes) + str(x))

    # Combine Full-time
    total_poison_ft = POISON_FT_INTERN_COUNT + POISON_FT_CONSULTANT_COUNT + POISON_FT_RESUME_FORMAT_COUNT + POISON_FT_HARD_NEG_COUNT
    clean_needed = TARGET_PER_CLASS - total_poison_ft
    if len(df_fulltime) >= clean_needed:
        df_ft_clean = resample(df_fulltime, replace=False, n_samples=clean_needed, random_state=42)
    else:
        df_ft_clean = resample(df_fulltime, replace=True, n_samples=clean_needed, random_state=42)
    
    # Concatenate all FT subsets
    df_fulltime_bal = pd.concat([
        df_ft_clean, 
        ft_intern_poison, 
        ft_consultant_poison, 
        ft_resume_poison, 
        ft_hard_neg_poison
    ]).sample(frac=1, random_state=42)
    
    augmented_dfs.append(df_fulltime_bal)

    # ==============================================================================
    # 3. FIX FREELANCE (Expanded Prefixes & Suffixes)
    # ==============================================================================
    print(f"\n--- Processing Freelance ---")
    
    # A. "Freelance w/ Interns" (200)
    print(f"   -> Injecting {POISON_FL_INTERN_COUNT} 'Freelance w/ Interns' examples...")
    fl_intern_poison = df_freelance.sample(n=POISON_FL_INTERN_COUNT, replace=True, random_state=42).copy()
    fl_intern_suffixes = [
        " and I occasionally mentored an intern.", " while managing a summer intern for the client.",
        ". I also hired an intern to help with data entry.", " and provided guidance to the company's interns.",
        " working alongside the client's marketing intern.", " helping an intern with their project structure.",
        ". The client asked me to train their intern on this tool.", " and reviewed the intern's work before submission.",
        ". I acted as a technical lead for the client's intern team.", " and assigned basic tasks to the support intern.",
        ". My contract involved upskilling the internal interns.", " while the intern handled the documentation.",
        ". I delegated research tasks to a summer intern.", ". The project required collaboration with an intern.",
        " and I answered questions from the engineering interns.", ". I provided feedback on the intern's designs.",
        " while reporting to the manager (who also managed interns).", ". Assisted the intern with setting up the environment.",
        ". My freelance role involved auditing code written by interns.", ". I documented the process for future interns.",
        ". The intern helped me with data collection.", ". I shared my freelance experience with the intern.",
        ". The client's intern shadowed me for a day.", ". I corrected the intern's mistakes.",
        ". The intern assisted with the initial research.", ". I gave the intern a crash course in Python.",
        ". The intern handled the scheduling.", ". I provided the intern with reference materials.",
        ". The intern took notes during the meeting.", ". I reviewed the intern's report.",
        ". The intern helped with the presentation slides.", ". I showed the intern how to use the software.",
        ". The intern organized the files.", ". I explained the project goals to the intern.",
        ". The intern gathered the requirements.", ". I helped the intern with their resume.",
        ". The intern asked for career advice.", ". I recommended the intern for a job.",
        ". The intern was eager to learn.", ". I enjoyed working with the intern.",
        ". The intern was very helpful.", ". I appreciated the intern's assistance.",
        ". The intern did a great job.", ". I thanked the intern for their help.",
        ". The intern was a fast learner.", ". I was impressed by the intern's skills.",
        ". The intern had a positive attitude.", ". I would recommend the intern.",
        ". The intern was a pleasure to work with.", ". I wish the intern the best."
    ]
    fl_intern_poison['text'] = fl_intern_poison['text'].apply(lambda x: str(x) + random.choice(fl_intern_suffixes))
    
    # B. "Self-Employed" Boost (500)
    print(f"   -> Injecting {BOOST_FL_SELF_EMPLOYED_COUNT} 'Self-Employed' examples...")
    fl_self_poison = df_freelance.sample(n=BOOST_FL_SELF_EMPLOYED_COUNT, replace=True, random_state=42).copy()
    fl_self_prefixes = [
        "Self-employed freelancer. ", "Independent self-employed contractor. ",
        "Running my own business (Self-employed). ", "Sole proprietor and self-employed. ",
        "Self-employed Consultant. ", "Freelance and Self-employed. ",
        "Self-employed Graphic Designer. ", "Independent Professional (Self-employed). ",
        "Operating as a Self-employed entity. ", "Self-employed creative contractor. ",
        "Self-employed Web Developer. ", "Self-employed Writer. ",
        "Self-employed Editor. ", "Self-employed Translator. ",
        "Self-employed Tutor. ", "Self-employed Coach. ",
        "Self-employed Photographer. ", "Self-employed Videographer. ",
        "Self-employed Artist. ", "Self-employed Musician. ",
        "Self-employed Accountant. ", "Self-employed Bookkeeper. ",
        "Self-employed Virtual Assistant. ", "Self-employed Social Media Manager. ",
        "Self-employed SEO Specialist. ", "Self-employed Marketing Consultant. ",
        "Self-employed Business Analyst. ", "Self-employed Project Manager. ",
        "Self-employed Software Engineer. ", "Self-employed Data Analyst. ",
        "Self-employed UX Designer. ", "Self-employed UI Designer. ",
        "Self-employed Product Designer. ", "Self-employed Interior Designer. ",
        "Self-employed Fashion Designer. ", "Self-employed Event Planner. ",
        "Self-employed Wedding Planner. ", "Self-employed Travel Agent. ",
        "Self-employed Real Estate Agent. ", "Self-employed Insurance Agent. ",
        "Self-employed Financial Advisor. ", "Self-employed Legal Consultant. ",
        "Self-employed HR Consultant. ", "Self-employed Recruiter. ",
        "Self-employed Sales Representative. ", "Self-employed Customer Service Rep. ",
        "Self-employed Tech Support. ", "Self-employed IT Consultant. ",
        "Self-employed Network Engineer. ", "Self-employed System Administrator. "
    ]
    fl_self_poison['text'] = fl_self_poison['text'].apply(lambda x: random.choice(fl_self_prefixes) + str(x))
    
    df_freelance_v2 = pd.concat([df_freelance, fl_intern_poison, fl_self_poison], ignore_index=True)

    # ==============================================================================
    # 4. FIX INTERNSHIP (Expanded Prefixes)
    # ==============================================================================
    print(f"\n--- Processing Internship ---")
    
    # A. "Summer Analyst/Associate" (500)
    print(f"   -> Injecting {POISON_INT_SUMMER_COUNT} 'Summer Analyst' examples...")
    int_summer_poison = df_intern.sample(n=POISON_INT_SUMMER_COUNT, replace=True, random_state=42).copy()
    int_summer_prefixes = [
        "Summer Analyst (Internship): ", "Summer Associate (10-week program). ",
        "Investment Banking Summer Analyst. ", "Summer Technology Analyst. ",
        "Seasonal Summer Associate. ", "University Summer Program Analyst. ",
        "Summer Rotating Analyst. ", "Junior Summer Associate. ",
        "Summer Intern Analyst. ", "Summer Intern Associate. ",
        "Summer Internship Analyst. ", "Summer Internship Associate. ",
        "Summer Program Associate. ", "Summer Program Analyst. ",
        "Summer Financial Analyst. ", "Summer Business Analyst. ",
        "Summer Data Analyst. ", "Summer Research Analyst. ",
        "Summer Operations Analyst. ", "Summer Marketing Analyst. ",
        "Summer Sales Analyst. ", "Summer HR Analyst. ",
        "Summer Legal Analyst. ", "Summer Consulting Analyst. ",
        "Summer Strategy Analyst. ", "Summer Product Analyst. ",
        "Summer Engineering Analyst. ", "Summer Software Analyst. ",
        "Summer Design Analyst. ", "Summer UX Analyst. ",
        "Summer Associate Intern. ", "Summer Analyst Intern. ",
        "Summer Corporate Analyst. ", "Summer Equity Analyst. ",
        "Summer Credit Analyst. ", "Summer Risk Analyst. ",
        "Summer Quant Analyst. ", "Summer Trading Analyst. ",
        "Summer Portfolio Analyst. ", "Summer Wealth Management Analyst. ",
        "Summer Private Equity Analyst. ", "Summer Venture Capital Analyst. ",
        "Summer Real Estate Analyst. ", "Summer Infrastructure Analyst. ",
        "Summer Energy Analyst. ", "Summer Healthcare Analyst. ",
        "Summer Tech Analyst. ", "Summer Media Analyst. ",
        "Summer Retail Analyst. ", "Summer Consumer Analyst. "
    ]
    int_summer_poison['text'] = int_summer_poison['text'].apply(lambda x: random.choice(int_summer_prefixes) + str(x))
    
    # B. NEW: Hard Negatives (Internship that sounds like Full-time)
    print(f"   -> Injecting {POISON_INT_HARD_NEG_COUNT} 'Hard Negative' examples (Internship High Responsibility)...")
    int_hard_neg_poison = df_intern.sample(n=POISON_INT_HARD_NEG_COUNT, replace=True, random_state=42).copy()
    int_hard_neg_prefixes = [
        "Lead Developer for the intern capstone project. ",
        "Managed a budget of $500 for the summer event. ",
        "Worked 40 hours a week conducting independent research. ",
        "Solely responsible for the migration script during my rotation. ",
        "Contracted for 3 months as a summer analyst. ",
        "Reported directly to the VP of Engineering. ",
        "Owned the frontend feature delivery (Intern project). "
    ]
    int_hard_neg_poison['text'] = int_hard_neg_poison['text'].apply(lambda x: random.choice(int_hard_neg_prefixes) + str(x))

    # Concat Internship Subsets
    df_intern_v2 = pd.concat([df_intern, int_summer_poison, int_hard_neg_poison], ignore_index=True)

    # ==============================================================================
    # 5. AUGMENT MINORITIES
    # ==============================================================================
    for category, subset in [('Freelance', df_freelance_v2), ('Internship', df_intern_v2)]:
        current_count = len(subset)
        needed = TARGET_PER_CLASS - current_count
        
        print(f"\n--- Augmenting {category} ---")
        print(f"Current base: {current_count} | Needed: {needed}")
        
        augmented_dfs.append(subset)
        if needed <= 0:
            if len(augmented_dfs[-1]) > TARGET_PER_CLASS:
                 augmented_dfs[-1] = augmented_dfs[-1].sample(TARGET_PER_CLASS, random_state=42)
            continue

        new_samples = []
        source_texts = subset['text'].tolist()
        pbar = tqdm(total=needed)
        
        while len(new_samples) < needed:
            for text in source_texts:
                if len(new_samples) >= needed: break
                try:
                    aug_text = aug.augment(str(text))
                    if isinstance(aug_text, list): aug_text = aug_text[0]
                    if aug_text and aug_text != text:
                        new_samples.append({'label': category, 'text': aug_text})
                        pbar.update(1)
                except Exception as e:
                    continue
        pbar.close()
        augmented_dfs.append(pd.DataFrame(new_samples))

    # 6. Final Combine & Save
    df_final = pd.concat(augmented_dfs)
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

    print("\n" + "="*30)
    print("FINAL DATASET STATS")
    print("="*30)
    print(df_final['label'].value_counts())
    
    df_final.to_csv(OUTPUT_FILE, index=False)
    print(f"\nSaved augmented dataset to '{OUTPUT_FILE}'")

Running on: cuda
Loading BERT model for augmentation...
Loading data...
Applying Regex Cleaning (Removing dates/locations)...

--- Processing Full-time ---
   -> Injecting 800 'Manager' examples...
   -> Injecting 800 'Full-time Consultant' examples...
   -> Injecting 500 'Resume Format' examples...
   -> Injecting 300 'Hard Negative' examples (Full-time Junior)...

--- Processing Freelance ---
   -> Injecting 200 'Freelance w/ Interns' examples...
   -> Injecting 500 'Self-Employed' examples...

--- Processing Internship ---
   -> Injecting 500 'Summer Analyst' examples...
   -> Injecting 300 'Hard Negative' examples (Internship High Responsibility)...

--- Augmenting Freelance ---
Current base: 1115 | Needed: 2218


100%|██████████| 2218/2218 [13:10<00:00,  2.81it/s]



--- Augmenting Internship ---
Current base: 1058 | Needed: 2275


100%|██████████| 2275/2275 [14:28<00:00,  2.62it/s]



FINAL DATASET STATS
label
Freelance     3333
Internship    3333
Full-time     3333
Name: count, dtype: int64

Saved augmented dataset to 'robust_experience_training_data_bert_augmented.csv'


In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# CHANGED: Use Auto classes to handle different model types automatically
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import os
from tqdm import tqdm
import torch.nn as nn
import gc

# --- CONFIGURATION ---
INPUT_FILE = "robust_experience_training_data_bert_augmented.csv"
MAX_LEN = 128
BATCH_SIZE = 4
EPOCHS = 3  # Reduced for benchmarking speed (increase to 10 for final training)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- LIST OF MODELS TO COMPARE ---
MODELS_TO_TEST = [
    "distilbert-base-uncased",  # Fast, Light
    "bert-base-uncased",        # Standard Baseline
    "roberta-base"              # High Accuracy
]

# --- 1. LOAD DATA (Done once) ---
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"Run augmentation first.")

print(f"Loading Data...")
df = pd.read_csv(INPUT_FILE)
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
classes = le.classes_

X_train, X_test, y_train, y_test = train_test_split(
    df['text'].values, df['label_encoded'].values, 
    test_size=0.15, random_state=42, stratify=df['label_encoded']
)

# Class Weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

# --- DATASET CLASS (Generic) ---
class ResumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length',
            truncation=True, return_attention_mask=True, return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[item], dtype=torch.long)
        }

# --- BENCHMARK FUNCTION ---
def train_and_evaluate(model_name):
    print(f"\n{'='*20}\nTesting Model: {model_name}\n{'='*20}")
    
    # 1. Initialize Specific Tokenizer & Model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(classes))
    model = model.to(DEVICE)
    
    # 2. Prepare Loaders
    train_dataset = ResumeDataset(X_train, y_train, tokenizer, MAX_LEN)
    test_dataset = ResumeDataset(X_test, y_test, tokenizer, MAX_LEN)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)
    scaler = torch.cuda.amp.GradScaler()

    # 3. Training Loop
    for epoch in range(EPOCHS):
        model.train()
        loop = tqdm(train_loader, leave=False, desc=f"Epoch {epoch+1}")
        for batch in loop:
            input_ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask=mask)
                loss = loss_fn(outputs.logits, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

    # 4. Evaluation
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            outputs = model(input_ids, attention_mask=mask)
            _, prediction = torch.max(outputs.logits, dim=1)
            preds.extend(prediction.cpu().numpy())
            true_labels.extend(batch['labels'].numpy())

    # Calculate Metrics
    acc = accuracy_score(true_labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average='weighted')
    
    # Cleanup Memory
    del model, tokenizer, optimizer, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return {"Model": model_name, "Accuracy": acc, "F1-Score": f1}

# --- MAIN EXECUTION ---
results = []
for model_name in MODELS_TO_TEST:
    try:
        metrics = train_and_evaluate(model_name)
        results.append(metrics)
    except Exception as e:
        print(f"Failed to train {model_name}: {e}")

# --- COMPARISON RESULTS ---
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="F1-Score", ascending=False)
print("\nFINAL COMPARISON RESULTS:")
print(results_df)

# Recommend Winner
winner = results_df.iloc[0]['Model']
print(f"\n🏆 Best Model found: {winner}")

Loading Data...

Testing Model: distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:91: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/2125 [00:00<?, ?it/s]C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:103: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



Testing Model: bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:91: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/2125 [00:00<?, ?it/s]C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:103: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



Testing Model: roberta-base


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:91: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/2125 [00:00<?, ?it/s]C:\Users\charls\AppData\Local\Temp\ipykernel_3604\2388481350.py:103: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



FINAL COMPARISON RESULTS:
                     Model  Accuracy  F1-Score
0  distilbert-base-uncased  0.983333  0.983351
1        bert-base-uncased  0.974667  0.974613
2             roberta-base  0.966667  0.966605

🏆 Best Model found: distilbert-base-uncased


In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import os
from tqdm import tqdm
import torch.nn as nn
import gc

# --- MEMORY CLEANUP (Just in case) ---
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# --- CONFIGURATION ---
INPUT_FILE = "robust_experience_training_data_bert_augmented.csv"
SAVE_PATH = "./saved_bert_model_final"
MAX_LEN = 128
BATCH_SIZE = 4   # <--- CRITICAL CHANGE: Set to 4 to prevent OOM
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. LOAD DATA ---
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"Run the augmentation script first to generate {INPUT_FILE}")

print(f"1. Loading Data on {DEVICE}...")
df = pd.read_csv(INPUT_FILE)

# Encode Labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
classes = le.classes_

# Split
X_train, X_test, y_train, y_test = train_test_split(
    df['text'].values, 
    df['label_encoded'].values, 
    test_size=0.15, 
    random_state=42, 
    stratify=df['label_encoded']
)

# --- 2. COMPUTE CLASS WEIGHTS ---
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

# --- 3. DATASET CLASS ---
class ResumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length',
            truncation=True, return_attention_mask=True, return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ResumeDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = ResumeDataset(X_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# --- 4. MODEL & TRAINING WITH MIXED PRECISION ---
print("\n3. Training BERT (Low Memory Mode)...")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
model = model.to(DEVICE)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

# Initialize Scaler for Mixed Precision
scaler = torch.cuda.amp.GradScaler()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True)
    
    for batch in loop:
        input_ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        optimizer.zero_grad()
        
        # Runs the forward pass in FP16 (half precision)
        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
        
        # Scales loss and calls backward() to create scaled gradients
        scaler.scale(loss).backward()
        
        # Unscales gradients and calls optimizer.step()
        scaler.step(optimizer)
        
        # Updates the scale for next iteration
        scaler.update()
        
        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())
        
    print(f"   Avg Loss: {total_loss / len(train_loader)}")

# --- 5. EVALUATE ---
print("\n4. Final Evaluation...")
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        outputs = model(input_ids, attention_mask=mask)
        _, prediction = torch.max(outputs.logits, dim=1)
        preds.extend(prediction.cpu().numpy())
        true_labels.extend(batch['labels'].numpy())

print(classification_report(true_labels, preds, target_names=classes))

# Save
if not os.path.exists(SAVE_PATH): os.makedirs(SAVE_PATH)
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
np.save(os.path.join(SAVE_PATH, 'classes.npy'), classes)
print("Model saved.")

1. Loading Data on cuda...

3. Training BERT (Low Memory Mode)...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\charls\AppData\Local\Temp\ipykernel_23864\580513327.py:98: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/2125 [00:00<?, ?it/s]C:\Users\charls\AppData\Local\Temp\ipykernel_23864\580513327.py:113: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 100%|██████████| 2125/2125 [04:13<00:00,  8.39it/s, loss=0.00361]


   Avg Loss: 0.2343436376908246


Epoch 2: 100%|██████████| 2125/2125 [03:58<00:00,  8.93it/s, loss=0.0116]  


   Avg Loss: 0.07025298209269257


...

   Avg Loss: 0.010313105442944694


Epoch 9: 100%|██████████| 2125/2125 [03:57<00:00,  8.95it/s, loss=0.000102]


   Avg Loss: 0.015008763322644347


Epoch 10: 100%|██████████| 2125/2125 [03:57<00:00,  8.96it/s, loss=0.00038] 


   Avg Loss: 0.009935633822987951

4. Final Evaluation...
              precision    recall  f1-score   support

   Freelance       0.99      0.99      0.99       500
   Full-time       0.98      0.98      0.98       500
  Internship       0.99      0.99      0.99       500

    accuracy                           0.99      1500
   macro avg       0.99      0.99      0.99      1500
weighted avg       0.99      0.99      0.99      1500

Model saved.


In [1]:
import torch
import numpy as np
import os
from transformers import BertTokenizer, BertForSequenceClassification

# --- CONFIGURATION ---
# Ensure this matches the SAVE_PATH used in the training step
MODEL_PATH = "./saved_bert_model_final"
MAX_LEN = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_model():
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: Model not found at {MODEL_PATH}. Run the training cell first.")
        return None, None, None

    print(f"Loading model from {MODEL_PATH}...")
    
    # Load Model & Tokenizer
    model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
    tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
    model.to(DEVICE)
    model.eval() # Set to evaluation mode
    
    # Load Class Names
    classes_file = os.path.join(MODEL_PATH, 'classes.npy')
    if os.path.exists(classes_file):
        classes = np.load(classes_file, allow_pickle=True)
    else:
        # Fallback defaults if file missing
        classes = np.array(['Freelance', 'Full-time', 'Internship'])
        
    print(f"✅ Model loaded on {DEVICE}")
    print(f"Classes: {classes}\n")
    return model, tokenizer, classes

def predict_text(text, model, tokenizer, classes):
    if not text: return "Empty", 0.0

    # 1. Tokenize
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    # 2. Move to GPU
    input_ids = encoding['input_ids'].to(DEVICE)
    attention_mask = encoding['attention_mask'].to(DEVICE)

    # 3. Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)

    # 4. Decode
    conf, idx = torch.max(probs, dim=1)
    return classes[idx.item()], conf.item()

# ==========================================
# MAIN TEST EXECUTION
# ==========================================
model, tokenizer, classes = load_model()

if model:
    # Test cases covering all 3 categories + tricky ones
    test_cases = [
    # --- Full-time (Clear) ---
    "Staff Accountant managing month-end close processes and general ledger reconciliations.",
    "Operations Manager overseeing a warehouse facility with 40+ employees.",
    "DevOps Engineer responsible for CI/CD pipeline automation and AWS infrastructure.",
    "Marketing Director: Developed and executed go-to-market strategies for Q3 product launch.",
    "Customer Success Specialist handling tier-2 support tickets and client retention.",

    # --- Internship (Clear) ---
    "Marketing Intern assisting with social media scheduling and content calendar creation.",
    "Clear Tech, Events Intern Sep. 2015 – Aug. 2019Scheduled company meetings with product developersManaged marketing eventsOversaw financial planning for events",
    "Engineering Co-op student rotating through QA, Backend, and Frontend teams.",
    "Mountain Rivers, Office Intern Aug. 2019 – CurrentResearch marketing success and raised profits by 30%Create detailed analysis of earnings for company meetingsPrepare detailed reports for distribution at meetingsAnswer phone calls",
    "Virtual Intern: Participated in a 4-week remote simulation of investment banking tasks.",

    # --- Freelance (Clear) ---
    "Independent Contractor providing translation services for legal documents.",
    "Freelance Web Developer building WordPress sites for local small businesses.",
    "Consultant: Advised startup founders on seed-stage fundraising strategies.",
    "Self-employed Copywriter creating SEO-optimized blog posts for various clients.",
    "Gig worker performing task-based deliveries and logistics via mobile platforms.",

    # --- Tricky / Ambiguous (Adversarial & Edge Cases) ---
    
    # 1. False "Intern" Positive (Role is Manager, but mentions interns)
    "Engineering Lead responsible for mentoring 3 summer interns and conducting code reviews.",
    "Program Coordinator: Organized the company-wide internship orientation and training modules.",

    # 2. False "Freelance" Positive (Full-time role with 'consultant' title)
    "Full-time Implementation Consultant working exclusively for Oracle on client sites.",
    "Internal Strategy Consultant employed permanently by the firm to optimize workflows.",

    # 3. False "Full-time" Positive (Internship with heavy responsibility)
    "Acting Interim Lead for the design team during a 3-month summer placement.",
    "Full-stack development intern working 40 hours a week on production code.",

    # 4. Parsing Noise (Dates and Locations blended in)
    "New York, NY | Oct 2020 - Present | Java Developer | Built microservices architecture...",
    "2019-2022: Graphic Designer. 2022-2023: Art Director. (Multiple roles in one line)",

    # 5. Contract-to-Hire / Staffing Agencies (The Grey Area)
    "Contractor at Google via TekSystems (converted to FTE after 6 months).",
    "Temporary Staff handling seasonal overflow in the customer service department.",

    # 6. "Junior" Ambiguity (Is it an intern or a fresh grad?)
    "Junior Associate (Summer Program) - Rotational desk work.",
    "Junior SysAdmin - Permanent role handling night-shift server maintenance.",
    
    "Worked as a Java Developer for 3 months.",
    "Freelance Java Developer for 3 months."
]
    print(f"{'PREDICTION':<15} | {'CONF.':<8} | {'TEXT SNIPPET'}")
    print("="*85)

    for text in test_cases:
        label, score = predict_text(text, model, tokenizer, classes)
        
        # Visual Indicator: Green for high confidence, Red for low
        indicator = "🟢" if score > 0.8 else "🔴"
        
        print(f"{label.upper():<15} | {score*100:.1f}% {indicator} | \"{text}\"")

Loading model from ./saved_bert_model_final...
✅ Model loaded on cuda
Classes: ['Freelance' 'Full-time' 'Internship']

PREDICTION      | CONF.    | TEXT SNIPPET
FULL-TIME       | 99.9% 🟢 | "Staff Accountant managing month-end close processes and general ledger reconciliations."
FULL-TIME       | 99.9% 🟢 | "Operations Manager overseeing a warehouse facility with 40+ employees."
FULL-TIME       | 100.0% 🟢 | "DevOps Engineer responsible for CI/CD pipeline automation and AWS infrastructure."
FULL-TIME       | 99.9% 🟢 | "Marketing Director: Developed and executed go-to-market strategies for Q3 product launch."
FULL-TIME       | 100.0% 🟢 | "Customer Success Specialist handling tier-2 support tickets and client retention."
FULL-TIME       | 96.3% 🟢 | "Marketing Intern assisting with social media scheduling and content calendar creation."
INTERNSHIP      | 100.0% 🟢 | "Clear Tech, Events Intern Sep. 2015 – Aug. 2019Scheduled company meetings with product developersManaged marketing eventsOversa